In [1]:
from dash import Dash, html
import dash_leaflet as dl

from database import Base, FeatureSet, Feature, Colormap, Style, connect_db
from sqlalchemy import func
from shapely.wkb import loads
from shapely.geometry import shape, mapping
from tqdm import tqdm

Marker attributes:
```
alt, attribution, autoPan, autoPanOnFocus, autoPanPadding, autoPanSpeed, bubblingMouseEvents, children, clickData, dblclickData, disableDefaultEventHandlers, draggable, eventHandlers, icon, id, interactive, keyboard, loading_state, n_clicks, n_dblclicks, opacity, pane, position, riseOffset, riseOnHover, shadowPane, title, zIndexOffset
```

In [10]:
def style_to_dict(style):
    """
    Convert a Style from the database to a dictionary that can be used by dash-leaflet
    """
    style_dict = {
        'color': style.color,
        'fillColor': style.fill_color,
        'weight': style.line_weight
    }

    return style_dict

In [2]:
def create_marker(feature, style=None, popup=None) -> dl.Marker:

    geometry = feature.geometry

    x, y = func.ST_X(Feature.geometry), func.ST_Y(Feature.geometry)
    position = (y, x) # Leaflet uses (lat, lon)

    children = []

    if style is not None:
        icon = style.icon_name
        icon_prefix = style.icon_prefix
        color = style.color

    if popup is not None:
        children.append(dl.Popup(content=popup))

    marker = dl.Marker(
        position=position,
        children=children
        )

    return marker

In [3]:
def create_geojson(feature, style=None, popup=None) -> dl.GeoJSON:

    children = []

    if popup is not None:
        children.append(dl.Popup(content=popup))
    
    if style is not None:
        style_dict = style_to_dict(style)

    return dl.GeoJSON(
        data=feature,
        style=style_dict,
        children=children
        )

In [4]:
def feature_to_geojson(feature):
    """
    Convert a Feature from the database to a GeoJSON Feature object
    """

    properties = feature.properties
    geometry_type = feature.geometry_type
    feature_set = feature.feature_set

    raw_geometry = feature.geometry.data
    shape_geometry = loads(bytes(raw_geometry))
    geojson_geometry = mapping(shape_geometry)

    geojson =  {
        "type": "Feature",
        "geometry": geojson_geometry,
        "properties": properties
    }

    return geojson

In [8]:
app = Dash(
    # external_stylesheets=[
    #     'https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.4/css/all.min.css'
    # ]
)

app.layout = html.Div([
    dl.Map([
        dl.LayersControl(
            [
                dl.BaseLayer(
                    dl.TileLayer(),
                    name='OpenStreetMap',
                    checked=True
                    )
            ],
            id='layer_control'
        )
    ], zoom=12, center=(53.55, 9.99), style={'width': '1000px', 'height': '500px'}),
    html.Div(id="log")
])

In [9]:
# connect to the database
engine, session = connect_db()

# get the layers control
# html.Div > dl.Map > dl.LayersControl
layers_control = app.layout.children[0].children[0]

# get all feature_sets
# feature_sets = session.query(FeatureSet).all()

# exclude the FeatureSet with the name Straße, very big dataset
feature_sets = session.query(FeatureSet).filter(FeatureSet.name != "Straße").all()

for feature_set in tqdm(feature_sets):

    # get the features and style for this feature set
    features = feature_set.features
    style = feature_set.style

    if style is None:
        print("No style for feature set " + feature_set.name + ". Skipping...")
        continue

    # save all map objects of this feature_set in here
    layer_group_children = []

    # get the popup properties of this feature
    popup_properties = style.popup_properties

    for feature in features:

        # get the properties for the popup
        popup_content = f"<b>{feature_set.name}</b><br>"

        for property in popup_properties:
            current_property = popup_properties[property]
            value = feature.properties.get(current_property, '')
            popup_content += f"<b>{property}</b>: {value}<br>"

        geometry_type = feature.geometry_type

        if geometry_type.upper() == "Point":

            # create a marker
            marker = create_marker(feature, style, popup=popup_content)
            layer_group_children.append(marker)
        
        else:

            # create a geojson map object, shows a polygon
            geojson_dict = feature_to_geojson(feature)
            geojson = create_geojson(geojson_dict, style, popup=popup_content)
            layer_group_children.append(geojson)
    
    # create a layer group for this feature set
    layer_group = dl.LayerGroup(children=layer_group_children, id=feature_set.name)
    overlay = dl.Overlay(layer_group, name=feature_set.name)
    layers_control.children.append(overlay)
        
if __name__ == '__main__':
    app.run_server(debug=True)

100%|██████████| 15/15 [00:01<00:00,  8.50it/s]
